In [ ]:
import numpy as np
import pygame
import random

class Tetris:

    def __init__(self, rows, cols, max_tiles, random_seed, agent):
        self.rows = rows
        self.cols = cols
        self.max_tiles = max_tiles
        self.random_seed = random_seed
        self.agent = agent
        
        # Create table for game board, entries 1 means occupied, entries -1 means free
        # Use type float32 to simplify conversion to tensors in torch
        self.board = np.empty((rows, cols), dtype=np.float32)
        self.current_tile = -1
        self.tile_x = -1
        self.tile_y = -1
        self.tile_orientation = -1

        # Tile set (at most 2 by 2)
        #   x       
        #   x      x x

        #   0 x    x 0
        #   x 0    0 x

        #   x x    x 0    0 x    x x
        #   x 0    x x    x x    0 x

        #   x x
        #   x x
        
        # Tile structure of tiles[i, j]: 
        # The first dimension denotes x value, 
        # the length denotes the number of columns taken by the tile. 
        # The second dimension consist of pairs giving the y range: 
        # first element in the pair is the first row of the tile and 
        # the second element second is the last row plus 1 of the tile 
        # for the current column
        self.tiles = [
            [
                [[0, 2]], 
                [[0, 1], [0, 1]],
            ],
            [
                [[0, 1], [1, 2]], 
                [[1, 2], [0, 1]],
            ],
            [
                [[0, 2], [1, 2]], 
                [[0, 2], [0, 1]], 
                [[0, 1], [0, 2]], 
                [[1, 2], [0, 2]],
            ],
            [
                [[0, 2], [0, 2]],
            ],
        ]

        if random_seed:
            random.seed(random_seed)

        if self.agent is not None:
            self.agent.init_board(self)

        self.start()

    def start(self):
        self.gameover = False
        self.tile_count = 0
        self.board.fill(-1)
        self.new_tile()

    def new_tile(self):
        if self.tile_count < self.max_tiles:
            self.current_tile = random.randint(0, len(self.tiles) - 1)
            self.tile_count += 1
        else:
            self.gameover = True
        self.tile_x = self.cols // 2
        self.tile_y = self.rows
        self.tile_orientation = 0
        '''self.agent.get_state()'''

    def fn_check_boundary(self):
        for xLoop in range(len(self.tiles[self.current_tile][self.tile_orientation])):
            curx=self.tile_x+xLoop
            if(curx<0)or(curx>self.cols-1):
                return 1
        return 0

    def fn_move(self,new_tile_x,new_tile_orientation):
        if new_tile_orientation>=len(self.tiles[self.current_tile]):
            return 1
        old_tile_x=self.tile_x
        old_tile_orientation=self.tile_orientation
        self.tile_x=new_tile_x
        self.tile_orientation=new_tile_orientation
        if self.fn_check_boundary():
            self.tile_x=old_tile_x
            self.tile_orientation=old_tile_orientation
            return 1
        return 0

    def fn_drop(self):
        curtile=self.tiles[self.current_tile][self.tile_orientation]
        # Find first location where the piece collides with occupied locations on the game board
        self.tile_y=0
        for xLoop in range(len(curtile)):
            curx=(self.tile_x+xLoop)%self.cols
            # Find first occupied location in this column            
            cury=-1;
            for yLoop in range(self.rows-1,-1,-1):
                if self.board[yLoop,curx]>0:
                    # Calculate the y position for this column if no other columns are taken into account
                    cury=yLoop+1-curtile[xLoop][0]
                    break
            # Use the largest y position for all columns of the tile
            if self.tile_y<cury:
                self.tile_y=cury

        # Change board entries at the newly placed tile to occupied
        for xLoop in range(len(curtile)):
            if self.tile_y+curtile[xLoop][1]>self.rows:
                self.gameover=1
                return -100;
            else:
                self.board[self.tile_y+curtile[xLoop][0]:self.tile_y+curtile[xLoop][1],(xLoop+self.tile_x)%self.cols]=1

        # Remove full lines
        lineCount=0
        for yLoop in range(self.rows-1,-1,-1):
            if np.sum(np.array(self.board[yLoop,:])>0)==self.cols:
                lineCount+=1
                for y1Loop in range(yLoop,self.rows-1):
                    self.board[y1Loop,:]=self.board[y1Loop+1,:]
                self.board[self.rows-1,:]=-1
        if lineCount>0:
            curReward=10**(lineCount-1)
        else:
            curReward=0
        # Choose the next tile
        self.new_tile()

        return curReward

In [ ]:
class HumanPlayer:
    def init_board(self, tetris):
        self.episode=0
        self.reward_tots=[0]
        self.tetris=tetris

    def get_state(self):
        pass

    def next_turn(self,pygame):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                raise SystemExit("Game terminated")
            if event.type==pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.reward_tots=[0]
                    self.tetris.start()
                if not self.tetris.gameover:
                    if event.key == pygame.K_UP:
                        self.tetris.fn_move(self.tetris.tile_x,(self.tetris.tile_orientation+1)%len(self.tetris.tiles[self.tetris.current_tile]))
                    if event.key == pygame.K_LEFT:
                        self.tetris.fn_move(self.tetris.tile_x-1,self.tetris.tile_orientation)
                    if event.key == pygame.K_RIGHT:
                        self.tetris.fn_move(self.tetris.tile_x+1,self.tetris.tile_orientation)
                    if (event.key == pygame.K_DOWN) or (event.key == pygame.K_SPACE):
                        self.reward_tots[self.episode]+=self.tetris.fn_drop()


#play the game, use arrow keys to play
you = HumanPlayer()

In [ ]:
tetris = Tetris(rows=4, cols=4, max_tiles=50, random_seed=123456, agent=you)

In [ ]:

def Evaluate(tetris,agent,agent_evaluate=None,strategy_file='',evaluate_agent=True):
    if agent_evaluate is None:
        evaluate_agent=False

    if evaluate_agent:
        agent_evaluate.epsilon=0
        agent_evaluate.init_board(tetris)
        agent_evaluate.load_strategy(strategy_file)

    if isinstance(tetris.agent,HumanPlayer):
        # Define some colors for painting
        COLOR_BLACK = (0, 0, 0)
        COLOR_GREY = (128, 128, 128)
        COLOR_WHITE = (255, 255, 255)
        COLOR_RED =  (255, 0, 0)

        # Initialize the game engine
        pygame.init()
        screen=pygame.display.set_mode((200+tetris.cols*20,150+tetris.rows*20))
        clock=pygame.time.Clock()
        pygame.key.set_repeat(300,100)
        pygame.display.set_caption('Turn-based tetris')
        font=pygame.font.SysFont('Calibri',25,True)
        fontLarge=pygame.font.SysFont('Calibri',50,True)
        framerate=0;

        # Loop until the window is closed
        while True:
            if isinstance(tetris.agent,HumanPlayer):
                tetris.agent.next_turn(pygame)
            else:
                pygame.event.pump()
                for event in pygame.event.get():
                    if event.type==pygame.KEYDOWN:
                        if event.key==pygame.K_SPACE:
                            if framerate > 0:
                                framerate=0
                            else:
                                framerate=10
                        if (event.key==pygame.K_LEFT) and (framerate>1):
                            framerate-=1
                        if event.key==pygame.K_RIGHT:
                            framerate+=1
                tetris.agent.next_turn()

            if evaluate_agent:
                agent_evaluate.get_state()
                agent_evaluate.set_action()

            if pygame.display.get_active():
                # Paint game board
                screen.fill(COLOR_WHITE)

                for i in range(tetris.rows):
                    for j in range(tetris.cols):
                        pygame.draw.rect(screen,COLOR_GREY,[100+20*j,80+20*(tetris.rows-i),20,20],1)
                        if tetris.board[i][j] > 0:
                            pygame.draw.rect(screen,COLOR_BLACK,[101+20*j,81+20*(tetris.rows-i),18,18])

                if tetris.current_tile is not None:
                    curTile=tetris.tiles[tetris.current_tile][tetris.tile_orientation]
                    for xLoop in range(len(curTile)):
                        for yLoop in range(curTile[xLoop][0],curTile[xLoop][1]):
                            pygame.draw.rect(screen,COLOR_RED,[101+20*((xLoop+tetris.tile_x)%tetris.cols),81+20*(tetris.rows-(yLoop+tetris.tile_y)),18,18])

                screen.blit(font.render("Reward: "+str(agent.reward_tots[agent.episode]),True,COLOR_BLACK),[0,0])
                screen.blit(font.render("Tile "+str(tetris.tile_count)+"/"+str(tetris.max_tiles),True,COLOR_BLACK),[0,20])
                if framerate>0:
                    screen.blit(font.render("FPS: "+str(framerate),True,COLOR_BLACK),[320,0])
                screen.blit(font.render("Reward: "+str(agent.reward_tots[agent.episode]),True,COLOR_BLACK),[0,0])
                if tetris.gameover:
                    screen.blit(fontLarge.render("Game Over", True,COLOR_RED), [80, 200])
                    screen.blit(font.render("Press ESC to try again", True,COLOR_RED), [85, 265])

                pygame.display.flip()
                clock.tick(framerate)

Evaluate(tetris, you)